In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from scipy import ndimage
from sklearn.model_selection import train_test_split

## **Data Analysis**

In [2]:
labels_df = pd.read_csv("../data/train_info.csv")
labels_df

,file_name,hamiltonian
0,graph4443.png,yes
1,graph1905.png,no
2,graph7719.png,no
3,graph4902.png,no
4,graph4114.png,yes
...,...,...
4423,graph4672.png,yes
4424,graph3437.png,yes
4425,graph3649.png,no
4426,graph893.png,yes


In [3]:
def rgb2gray(rgb):
    img = np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])
    return img / 255

def convert_to_array(data_dir, labels):
    files = os.listdir(data_dir)
    dataset_x = []
    dataset_y = []
    for picture in files:
        img = cv2.imread(data_dir + f"/{picture}")
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (100, 100))
        img = rgb2gray(img)
        
        img = img.reshape(1, 100, 100)
        
        label = labels[labels["file_name"] == f'{picture}']["hamiltonian"].to_list()
        
        dataset_x.append(img)
        dataset_y.append(label[0])
    
    return np.array(dataset_x), dataset_y

data_train = convert_to_array("../data/train_images/train_images", labels_df)

In [4]:
images, labels = data_train

In [5]:
images_train, images_test, labels_train, labels_test = train_test_split(images, labels, test_size=0.2, random_state=123)

In [6]:
images_train.shape, images_test.shape

((3542, 1, 100, 100), (886, 1, 100, 100))

In [7]:
def plot_random_sample(images, label):
    random_indices = np.random.choice(images.shape[0], 12, replace=False)
    plt.figure(figsize=(20, 10), dpi=200)
    for i, idx in enumerate(random_indices):
        plt.subplot(3, 4, i + 1)
        plt.imshow(images[idx], cmap='gray')
        plt.title(f"Hamiltonian = {label.iloc[idx]['hamiltonian']}")
        plt.axis('on')
        plt.xticks([])
        plt.yticks([])
        plt.tight_layout()

# plot_random_sample(images_train, labels_df)

In [8]:
def augment_data(images, labels, num_augmentations):
    augmented_images = []
    augmented_labels = []

    for i in range(images.shape[0]):
        for _ in range(num_augmentations):
            angle = np.random.uniform(-90, 90)
            rotated_image = ndimage.rotate(images[i], angle, reshape=True)

            shift = np.random.uniform(-5, 5, 2)
            shifted_image = ndimage.shift(rotated_image, shift)

            if np.random.rand() > 0.5:
                flipped_image = np.fliplr(shifted_image)
            else:
                flipped_image = shifted_image

            augmented_images.append(flipped_image)
            augmented_labels.append(labels.iloc[i])

    max_shape = np.max([img.shape for img in augmented_images], axis=0)

    resized_images = []
    for img in augmented_images:
        resized_img = cv2.resize(img, (max_shape[1], max_shape[0]), interpolation=cv2.INTER_AREA)
        resized_images.append(resized_img)

    return np.array(resized_images), pd.DataFrame(augmented_labels)

In [9]:
# augmented_data = augment_data(images, label["hamiltonian"], 2)

## **Moedel Design**

In [10]:
import torch
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchvision import transforms, utils
import torch.nn as nn
import torch.nn.functional as F

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [11]:
if torch.cuda.is_available():
    print("CUDA is available!")
else:
    print("CUDA is not available.")
    
device_name = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_name)

CUDA is available!


In [12]:
labels_train = [[1.0] if i == "yes" else [0.0] for i in labels_train]
labels_test = [[1.0] if i == "yes" else [0.0] for i in labels_test]

In [13]:
torch_images_train = torch.from_numpy(images_train).type(torch.float).to(device)
torch_labels_train = torch.from_numpy(np.array(labels_train)).type(torch.float).to(device)

torch_images_test = torch.from_numpy(images_test).type(torch.float).to(device)
torch_labels_test = torch.from_numpy(np.array(labels_test)).type(torch.float).to(device)

torch_images_train.size(), torch_labels_train.size(), torch_images_test.size(), torch_labels_test.size()

(torch.Size([3542, 1, 100, 100]),
 torch.Size([3542, 1]),
 torch.Size([886, 1, 100, 100]),
 torch.Size([886, 1]))

In [54]:
dataset_train = TensorDataset(torch_images_train, torch_labels_train)
dataset_test = TensorDataset(torch_images_test, torch_labels_test)

dataloader_train = DataLoader(
    dataset_train, 
    batch_size=32,
    # shuffle=True
)

dataloader_test = DataLoader(
    dataset_test, 
    batch_size=32,
    # shuffle=True
)

In [55]:
for i, sample in enumerate(dataloader_train):
    if i == 0:
        res = sample
    else:
        break

In [56]:
res[0].size(), res[1].size()

(torch.Size([32, 1, 100, 100]), torch.Size([32, 1]))

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        self.conv1 = nn.Conv2d(
            in_channels=1,
            out_channels=64,
            kernel_size=6,
            stride=1,
            padding="same"
        ) 
        self.batch_norm_1 = nn.BatchNorm2d(
            num_features=64
        )

        self.conv2 = nn.Conv2d(
            in_channels=64, 
            out_channels=128,
            kernel_size=16, 
            stride=1,
            padding="same"
        )
        self.batch_norm_2 = nn.BatchNorm2d(
            num_features=128
        )
        
        self.dropout_1 = nn.Dropout2d(p=0.2)
        self.dropout_2 = nn.Dropout(p=0.2)
        
        self.fc1 = nn.Linear(
            in_features=80000, 
            out_features=128
        )
        self.fc2 = nn.Linear(
            in_features=128, 
            out_features=1
        )

    
    def forward(self, x):
        # print(x.size())
        x = self.conv1(x)
        x = F.relu(x)
        x = self.batch_norm_1(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = self.dropout_1(x)
        
        # print(x.size())
        
        x = self.conv2(x)
        x = F.relu(x)
        x = self.batch_norm_2(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout_1(x)
        
        x = F.
        
        # print(x.size())
        
        x = x.view(-1, 80000)
        # print(x.size())
        
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout_2(x)
        
        # print(x.size())
        x = self.fc2(x)
        output = F.sigmoid(x)
        # print(output.size())
        # x = self.dropout_2(x)
        # print(output.size())
        return output

In [63]:
model = Model()
model.to(device)

Model(
  (conv1): Conv2d(1, 64, kernel_size=(6, 6), stride=(1, 1), padding=same)
  (batch_norm_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 128, kernel_size=(16, 16), stride=(1, 1), padding=same)
  (batch_norm_2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout_1): Dropout2d(p=0.2, inplace=False)
  (dropout_2): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=80000, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
)

In [64]:
# optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=0.001
)
loss_fn = nn.BCELoss()

In [65]:
def _evaluate(
    valid_loader: DataLoader,
    loss_function: nn.modules.loss._Loss
) -> float:

    model.eval()
    valid_loss = 0.0
    total_correct = 0
    
    with torch.no_grad():
        for batch_data_valid, batch_target_valid in dataloader_test:
            output_valid = model(batch_data_valid)
            loss_valid = loss_function(output_valid, batch_target_valid)
            valid_loss += loss_valid.item() * batch_data_valid.size(0)
            
            predicted = torch.where(output_valid < 0.5, 0.0, 1.0)
            total_correct += (predicted == batch_target_valid).sum().item()
            
    valid_accuracy = 100 * total_correct / len(dataloader_test.dataset)
    valid_loss = valid_loss / len(valid_loader.dataset)
    return valid_loss, valid_accuracy

In [66]:
train_loss_min = np.Inf
valid_loss_min = np.Inf
last_valid_loss = 0
freq = 1
n_epochs = 50

for epoch in range(1, n_epochs + 1):
    model.train()
    train_loss = 0.0
    total_correct = 0
    
    for batch_data_train, batch_target_train in dataloader_train:
        optimizer.zero_grad()
        output = model(batch_data_train)

        predicted = torch.where(output < 0.5, 0.0, 1.0)
        
        # print(predicted)
        
        loss = loss_fn(output, batch_target_train)
        loss.backward()
        optimizer.step()
        
        
        train_loss += loss.item() * batch_data_train.size(0)
        
        total_correct += (predicted == batch_target_train).sum().item()
    
    train_accuracy = 100 * total_correct / len(dataloader_train.dataset)
    train_loss = train_loss / len(dataloader_train.dataset)
    valid_loss, valid_accuracy = _evaluate(dataloader_test, loss_fn)
        
    if valid_loss == last_valid_loss:
        print('problem')
    
    last_valid_loss = valid_loss
    if epoch % freq == 0:
        print(f'Epoch [{epoch}/{n_epochs}] \tTraining Loss: {train_loss:.6f} \tValidation Loss: {valid_loss:.6f} \tTraining Accuracy: {train_accuracy}, \tValidation Accuracy: {valid_accuracy}')
        
        
    if valid_loss < valid_loss_min:
        print(f'Validation loss decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}).  Saving model ...')
        state_dict = model.state_dict()
        valid_loss_min = valid_loss


Epoch [1/50] 	Training Loss: 4.188256 	Validation Loss: 0.585431 	Training Accuracy: 70.4404291360813, 	Validation Accuracy: 68.51015801354401
Validation loss decreased (inf --> 0.585431).  Saving model ...
Epoch [2/50] 	Training Loss: 0.527493 	Validation Loss: 0.932218 	Training Accuracy: 76.84923771880294, 	Validation Accuracy: 59.706546275395034
Epoch [3/50] 	Training Loss: 0.442527 	Validation Loss: 1.250341 	Training Accuracy: 79.22077922077922, 	Validation Accuracy: 51.580135440180584
Epoch [4/50] 	Training Loss: 0.400262 	Validation Loss: 0.358286 	Training Accuracy: 82.10050818746471, 	Validation Accuracy: 83.52144469525959
Validation loss decreased (0.585431 --> 0.358286).  Saving model ...
Epoch [5/50] 	Training Loss: 0.375882 	Validation Loss: 1.209999 	Training Accuracy: 83.37097684923772, 	Validation Accuracy: 53.160270880361175
Epoch [6/50] 	Training Loss: 0.356054 	Validation Loss: 1.962069 	Training Accuracy: 85.62958780350084, 	Validation Accuracy: 50.33860045146727
E

KeyboardInterrupt: 